In [1]:
import pandas as pd
import numpy as np

# 1. Load CSV full_sensor_data to DataFrame

In [2]:
history = pd.read_csv("data/full_sensor_data.csv", parse_dates=True, index_col="UTC time")

## 1.1. Data group by 1Day

In [3]:
history_aggegated_1D = history.groupby(pd.Grouper(freq='1D')).mean()

### Creating MultiIndex

In [4]:
history_aggegated_1D.columns = pd.MultiIndex.from_tuples([(c.split('_')[0], c.split('_')[1]) for c in history_aggegated_1D.columns])

In [45]:
history_aggegated_1D.head()

147                                                    \
           temperature   humidity       pressure         pm1        pm25   
UTC time                                                                   
2017-01-01   -2.083333  45.750000  101996.458333  135.791667  149.250000   
2017-01-02   -2.333333  64.916667  101431.208333   73.833333   78.291667   
2017-01-03   -2.125000  76.125000  101544.875000   38.416667   37.041667   
2017-01-04   -0.750000  75.458333  100006.208333   12.583333    5.125000   
2017-01-05   -5.541667  54.250000  101636.916667   14.708333    8.041667   

                               169                                       ...  \
                  pm10 temperature   humidity       pressure        pm1  ...   
UTC time                                                                 ...   
2017-01-01  234.458333    2.500000  43.375000  102167.000000  89.750000  ...   
2017-01-02  133.416667    3.458333  47.375000  101589.333333  58.750000  ...   
2017-01-03   68.041667    4.291667  50.500000  101717.416667  25.791667  ...   
2017-01-04   14.708333    3.625000  55.791667  100157.500000   5.416667  ...   
2017-01-05   19.250000   -0.909091  46.090909  101839.636364   8.909091  ...   

                  187_808                                       211_622  \
                 pressure        pm1       pm25        pm10 temperature   
UTC time                                                                  
2017-01-01  101777.083333  84.000000  93.583333  151.791667    2.000000   
2017-01-02  101216.318182  52.227273  55.954545  100.136364    1.166667   
2017-01-03  101324.333333  29.500000  27.500000   52.166667    1.291667   
2017-01-04   99780.750000  10.500000   4.375000   13.916667    1.708333   
2017-01-05  101409.208333  13.500000   7.958333   19.666667   -1.291667   

                                                                         
             humidity       pressure        pm1        pm25        pm10  
UTC time                                                                 
2017-01-01  16.708333  102192.541667  96.041667  113.875000  177.791667  
2017-01-02  25.458333  101627.833333  53.333333   59.916667  103.375000  
2017-01-03  30.333333  101751.625000  28.333333   27.166667   50.583333  
2017-01-04  34.958333  100210.083333   9.291667    3.541667   11.875000  
2017-01-05  20.875000  101853.583333   9.083333    3.291667   11.833333  

[5 rows x 300 columns]

### Merging data from pairs of sensors with the similar locations

In [5]:
sensor_pairs = [('3','140'),('227','857'),('178','713'),('142','895'),('187','808'),('211','622')]
combinedes = pd.DataFrame()
for pair in sensor_pairs:
    combined = history_aggegated_1D[pair[0]].combine_first(history_aggegated_1D[pair[1]])
    combined.columns = pd.MultiIndex.from_product([[pair[0]+'_'+pair[1]],combined.columns])
    combinedes = pd.concat([combinedes,combined], axis=1)
history_aggegated_1D = pd.concat([history_aggegated_1D, combinedes], axis=1)
history_aggegated_1D = history_aggegated_1D.drop(columns=['3','140','227','857','178','713','142','895','187','808','211','622'], level=0)

### 1.1.1. Imputing using mean

In [6]:
filled_mean_history_aggegated_1D = history_aggegated_1D.fillna(history_aggegated_1D.mean())

### 1.1.2. Imputing using the meadian

In [7]:
filled_median_history_aggegated_1D = history_aggegated_1D.fillna(history_aggegated_1D.median())

### 1.1.3 Imputing using the rolling mean + imputing using mean for Nan for which rolling mean doesn't work

In [8]:
filled_rolling_mean_history_aggegated_1D = history_aggegated_1D.fillna(history_aggegated_1D.rolling(24, min_periods=1).mean())
filled_rolling_mean_history_aggegated_1D = filled_rolling_mean_history_aggegated_1D.fillna(filled_rolling_mean_history_aggegated_1D.mean())

### 1.1.4 Imputing using the rolling median + imputing using median for Nan for which rolling median doesn't work

In [9]:
filled_rolling_median_history_aggegated_1D = history_aggegated_1D.fillna(history_aggegated_1D.rolling(24, min_periods=1).median())
filled_rolling_median_history_aggegated_1D = filled_rolling_median_history_aggegated_1D.fillna(filled_rolling_median_history_aggegated_1D.median())

### 1.1.5 Imputing using the interpolation with linear method

In [10]:
filled_interpolation_linear_history_aggegated_1D = history_aggegated_1D.interpolate(method='linear')

### 1.1.6 Imputing using the interpolation with time method

In [11]:
filled_interpolation_time_history_aggegated_1D = history_aggegated_1D.interpolate(method='time')

### 1.1.7 Imputing using the interpolation with quadratic method

In [12]:
filled_interpolation_quadratic_history_aggegated_1D = history_aggegated_1D.interpolate(method='quadratic')

# Trenowanie modelu


In [13]:
history_filled = filled_median_history_aggegated_1D
history_filled.head()

147                                                    \
           temperature   humidity       pressure         pm1        pm25   
UTC time                                                                   
2017-01-01   -2.083333  45.750000  101996.458333  135.791667  149.250000   
2017-01-02   -2.333333  64.916667  101431.208333   73.833333   78.291667   
2017-01-03   -2.125000  76.125000  101544.875000   38.416667   37.041667   
2017-01-04   -0.750000  75.458333  100006.208333   12.583333    5.125000   
2017-01-05   -5.541667  54.250000  101636.916667   14.708333    8.041667   

                               169                                       ...  \
                  pm10 temperature   humidity       pressure        pm1  ...   
UTC time                                                                 ...   
2017-01-01  234.458333    2.500000  43.375000  102167.000000  89.750000  ...   
2017-01-02  133.416667    3.458333  47.375000  101589.333333  58.750000  ...   
2017-01-03   68.041667    4.291667  50.500000  101717.416667  25.791667  ...   
2017-01-04   14.708333    3.625000  55.791667  100157.500000   5.416667  ...   
2017-01-05   19.250000   -0.909091  46.090909  101839.636364   8.909091  ...   

                  187_808                                       211_622  \
                 pressure        pm1       pm25        pm10 temperature   
UTC time                                                                  
2017-01-01  101777.083333  84.000000  93.583333  151.791667    2.000000   
2017-01-02  101216.318182  52.227273  55.954545  100.136364    1.166667   
2017-01-03  101324.333333  29.500000  27.500000   52.166667    1.291667   
2017-01-04   99780.750000  10.500000   4.375000   13.916667    1.708333   
2017-01-05  101409.208333  13.500000   7.958333   19.666667   -1.291667   

                                                                         
             humidity       pressure        pm1        pm25        pm10  
UTC time                                                                 
2017-01-01  16.708333  102192.541667  96.041667  113.875000  177.791667  
2017-01-02  25.458333  101627.833333  53.333333   59.916667  103.375000  
2017-01-03  30.333333  101751.625000  28.333333   27.166667   50.583333  
2017-01-04  34.958333  100210.083333   9.291667    3.541667   11.875000  
2017-01-05  20.875000  101853.583333   9.083333    3.291667   11.833333  

[5 rows x 300 columns]

In [40]:
columns = history_filled.columns.to_list()
#print(columns)
sensors = list(set([str(column[0]) for column in history_filled.columns]))
#print(sensors)


In [16]:
training_data = history_filled
training_data_y = pd.DataFrame()

for s in sensors:
    training_data_y[str(s)+'_pm1_next_day'] = history_filled[str(s),'pm1'].shift(-1)
    training_data_y[str(s)+'_pm25_next_day'] = history_filled[str(s),'pm25'].shift(-1)
    training_data_y[str(s)+'_pm10_next_day'] = history_filled[str(s),'pm10'].shift(-1)

training_data_y.head()

,227_857_pm1_next_day,227_857_pm25_next_day,227_857_pm10_next_day,182_pm1_next_day,182_pm25_next_day,182_pm10_next_day,192_pm1_next_day,192_pm25_next_day,192_pm10_next_day,187_808_pm1_next_day,...,228_pm10_next_day,170_pm1_next_day,170_pm25_next_day,170_pm10_next_day,169_pm1_next_day,169_pm25_next_day,169_pm10_next_day,184_pm1_next_day,184_pm25_next_day,184_pm10_next_day
UTC time,,,,,,,,,,,,,,,,,,,,,
2017-01-01,49.625000,53.000000,95.125000,48.750000,51.833333,95.375000,58.708333,64.583333,107.375000,52.227273,...,101.000000,53.833333,59.791667,105.708333,58.750000,64.250000,112.208333,19.708333,17.875000,31.687500
2017-01-02,25.750000,22.583333,43.000000,30.500000,28.791667,54.125000,32.000000,30.833333,55.291667,29.500000,...,48.791667,28.458333,27.291667,53.041667,25.791667,23.416667,46.750000,19.708333,17.875000,31.687500
2017-01-03,7.666667,2.083333,8.541667,10.041667,4.083333,12.541667,10.708333,4.208333,11.958333,10.500000,...,11.916667,8.083333,2.125000,10.625000,5.416667,0.500000,6.250000,9.166667,2.000000,10.000000
2017-01-04,10.791667,5.250000,14.583333,11.166667,4.875000,14.625000,16.833333,11.750000,24.041667,13.500000,...,14.166667,9.875000,4.625000,14.458333,8.909091,3.863636,14.136364,8.250000,2.458333,8.041667
2017-01-05,20.875000,16.541667,33.333333,22.375000,17.666667,35.541667,36.208333,35.666667,60.833333,30.208333,...,37.083333,23.791667,21.375000,43.791667,16.750000,12.416667,28.791667,20.625000,14.500000,28.625000


In [17]:
training_data_x = pd.DataFrame()
for s in sensors:
    #print("sensor: " + s)
    for column in training_data[s].columns:
        #print("column: " + column)
        #print(training_data[s,column])
        training_data_x[s +"_"+column] = training_data[s,column]

training_data_x.head()

,227_857_temperature,227_857_humidity,227_857_pressure,227_857_pm1,227_857_pm25,227_857_pm10,182_temperature,182_humidity,182_pressure,182_pm1,...,169_pressure,169_pm1,169_pm25,169_pm10,184_temperature,184_humidity,184_pressure,184_pm1,184_pm25,184_pm10
UTC time,,,,,,,,,,,,,,,,,,,,,
2017-01-01,1.791667,32.833333,102078.875000,87.916667,99.166667,158.166667,1.708333,46.458333,101707.6875,83.791667,...,102167.000000,89.750000,101.833333,164.833333,8.895833,79.729167,101713.583333,19.708333,17.875000,31.687500
2017-01-02,0.916667,41.791667,101527.833333,49.625000,53.000000,95.125000,0.083333,58.666667,101707.6875,48.750000,...,101589.333333,58.750000,64.250000,112.208333,8.895833,79.729167,101713.583333,19.708333,17.875000,31.687500
2017-01-03,0.826087,47.173913,101645.625000,25.750000,22.583333,43.000000,0.458333,63.166667,101707.6875,30.500000,...,101717.416667,25.791667,23.416667,46.750000,8.895833,79.729167,101713.583333,19.708333,17.875000,31.687500
2017-01-04,1.190476,50.761905,100088.708333,7.666667,2.083333,8.541667,0.708333,66.791667,101707.6875,10.041667,...,100157.500000,5.416667,0.500000,6.250000,1.000000,65.666667,101713.583333,9.166667,2.000000,10.000000
2017-01-05,-2.375000,38.416667,101680.958333,10.791667,5.250000,14.583333,-2.750000,54.958333,101707.6875,11.166667,...,101839.636364,8.909091,3.863636,14.136364,-1.000000,48.583333,101713.583333,8.250000,2.458333,8.041667


In [18]:
from sklearn.preprocessing import StandardScaler

training_data_y_scaled  = StandardScaler().fit_transform(training_data_y)
training_data_y_scaled
scaled_features_y_df = pd.DataFrame(training_data_y_scaled, index=training_data_y.index, columns=training_data_y.columns)
scaled_features_y_df.head()

training_data_x_scaled  = StandardScaler().fit_transform(training_data_x)
training_data_x_scaled
scaled_features_x_df = pd.DataFrame(training_data_x_scaled, index=training_data_x.index, columns=training_data_x.columns)
scaled_features_x_df.head()

C:\Users\Grzegorz\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\Grzegorz\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\extmath.py:689: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


,227_857_temperature,227_857_humidity,227_857_pressure,227_857_pm1,227_857_pm25,227_857_pm10,182_temperature,182_humidity,182_pressure,182_pm1,...,169_pressure,169_pm1,169_pm25,169_pm10,184_temperature,184_humidity,184_pressure,184_pm1,184_pm25,184_pm10
UTC time,,,,,,,,,,,,,,,,,,,,,
2017-01-01,-0.961958,-3.092617,0.354400,2.582862,2.544904,2.328214,-0.858498,-2.482814,0.044845,1.863847,...,0.634854,2.645054,2.734963,2.998364,-0.042307,0.140517,0.060516,-0.335881,-0.346088,-0.331833
2017-01-02,-1.103576,-2.426356,-0.419980,0.917015,0.863547,0.942158,-1.071300,-1.508821,0.044845,0.618237,...,-0.108385,1.358782,1.380628,1.716974,-0.042307,0.140517,0.060516,-0.335881,-0.346088,-0.331833
2017-01-03,-1.118236,-2.026061,-0.254447,-0.121647,-0.244206,-0.203880,-1.022192,-1.149807,0.044845,-0.030488,...,0.056410,-0.008746,-0.090822,0.123100,-0.042307,0.140517,0.060516,-0.335881,-0.346088,-0.331833
2017-01-04,-1.059260,-1.759210,-2.442386,-0.908347,-0.990801,-0.961492,-0.989453,-0.860600,0.044845,-0.757712,...,-1.950611,-0.854158,-0.916635,-0.863053,-1.064027,-0.983337,0.060516,-0.644748,-0.789827,-0.752501
2017-01-05,-1.636331,-2.677366,-0.204793,-0.772396,-0.875474,-0.828658,-1.442338,-1.804675,0.044845,-0.717722,...,0.213661,-0.709248,-0.795425,-0.671024,-1.322827,-2.348611,0.060516,-0.671605,-0.777015,-0.790486


In [19]:
#tu troche oszukuje jak sa jakies nulle bo jest ich po kilkaset w kazdym przypadku

scaled_features_x_df = scaled_features_x_df.fillna(0)
scaled_features_x_df.isnull().sum().sum()

scaled_features_y_df = scaled_features_y_df.fillna(0)
scaled_features_y_df.isnull().sum().sum()

0

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_features_x_df, scaled_features_y_df, test_size=0.20, random_state=42)
X_train

,227_857_temperature,227_857_humidity,227_857_pressure,227_857_pm1,227_857_pm25,227_857_pm10,182_temperature,182_humidity,182_pressure,182_pm1,...,169_pressure,169_pm1,169_pm25,169_pm10,184_temperature,184_humidity,184_pressure,184_pm1,184_pm25,184_pm10
UTC time,,,,,,,,,,,,,,,,,,,,,
2017-10-17,0.939776,0.291369,0.767734,0.162943,0.074463,0.175385,1.078539,-0.476987,2.060673,-0.261541,...,0.865588,0.088070,0.021789,0.077445,0.510339,0.505146,1.856606,0.024259,-0.007170,0.071863
2017-12-02,-1.649819,1.177652,0.759537,2.160509,2.391639,3.389970,-1.480533,1.539477,1.877707,1.706850,...,0.830366,1.638858,1.535281,1.394344,-1.506143,1.524107,1.785295,1.657708,1.555815,1.469239
2017-08-20,-0.025542,0.222419,0.034108,-0.281162,-0.285178,-0.321140,0.642023,1.163842,0.044845,-0.844110,...,0.357748,-0.752155,-0.679402,-0.724058,0.531906,1.287681,0.060516,-0.686255,-0.659384,-0.717748
2017-03-17,-0.460673,-0.336672,-0.360138,0.204635,0.056253,-0.022492,-0.450549,-0.721962,0.044845,0.242384,...,-0.535543,0.181429,0.096863,0.152522,-0.427810,-0.796860,0.060516,-0.053873,-0.095684,-0.021080
2017-07-14,1.402396,-2.143118,0.146708,-0.766958,-0.778356,-0.855225,0.860281,-1.056729,0.044845,-0.618488,...,-0.069250,-0.600016,-0.574298,-0.617529,0.817664,-0.956697,0.060516,-0.613006,-0.599986,-0.648243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-03-13,-0.671977,-0.204453,1.364754,0.336960,0.236832,0.089272,-0.711175,-0.252271,0.044845,0.424212,...,1.058581,0.643034,0.584844,0.584725,-0.713569,-0.260740,0.060516,0.103612,0.074357,0.149450
2017-04-17,-0.463801,0.528502,-0.158594,-0.377234,-0.415680,-0.484205,-0.547481,0.312845,0.044845,-0.305974,...,-0.316924,-0.283634,-0.322049,-0.317220,-0.551819,0.348639,0.060516,-0.356635,-0.385687,-0.392044
2017-09-28,0.487946,-0.179662,1.690081,-0.215906,-0.282143,-0.219453,-0.094596,0.064637,0.044845,-0.305974,...,1.719636,-0.314754,-0.350578,-0.347656,0.159881,0.205451,0.060516,-0.305361,-0.329783,-0.309607


In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, make_scorer

mlp_reg = MLPRegressor(max_iter=1000)
mlp_reg.fit(X_train, y_train)
y_pred_mlp = mlp_reg.predict(X_test)
print('Coefficient of determination - mlp: %.2f' % r2_score(y_test, y_pred_mlp))


Coefficient of determination - mlp: 0.63


In [44]:
pol2_feature_trans = PolynomialFeatures(2).fit(X_train)
pol2_features = pol2_feature_trans.transform(X_train)
pol2_reg = Ridge(alpha=1.0, solver='lsqr').fit(pol2_features, y_train)

X_test_pol2 = pol2_feature_trans.transform(X_test)
y_pred_pol2 = pol2_reg.predict(X_test_pol2)
print('Coefficient of determination - polinomial 2: %.2f' % r2_score(y_test, y_pred_pol2))

Coefficient of determination - polinomial 2: 0.45


In [22]:
from sklearn.model_selection import GridSearchCV
reg = MLPRegressor()
scorers_pol2 = make_scorer(r2_score)

param_grid = {
    'alpha' :[0.0001],
    'solver' : ['lbfgs', 'sgd', 'adam'],
    'activation' : ['identity', 'logistic', 'tanh', 'relu'],
    
}

scorers_pol2 = GridSearchCV(estimator=reg, 
                     param_grid=param_grid, 
                     scoring=scorers_pol2, 
                     cv=5)

scorers_pol2.fit(X=scaled_features_x_df, y=scaled_features_y_df)
scorers_pol2.best_params_

C:\Users\Grzegorz\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\Grzegorz\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\Grzegorz\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarni

C:\Users\Grzegorz\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\Grzegorz\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\Grzegorz\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarni

C:\Users\Grzegorz\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'activation': 'relu', 'alpha': 0.0001, 'solver': 'sgd'}

In [39]:
from sklearn import metrics

clf = MLPRegressor(alpha  = 0.0001, solver = 'sgd', activation ='relu')
clf.fit(X_train,y_train)
predicted = clf.predict(X_test)
clf.score(X_test, predicted)

1.0